### Interactive Fluorescent Segmentation Code

Use this notebook to figure out the segmentation hyperparameters that are most appropriate for your dataset.

In [ ]:
import trenchripper as tr

from ipywidgets import (
    interact,
    interactive,
    fixed,
    interact_manual,
    FloatSlider,
    IntSlider,
    Dropdown,
    IntText,
    IntRangeSlider,
)
import ipywidgets as widgets
import matplotlib
import warnings
import copy

matplotlib.rcParams["figure.figsize"] = [20, 10]
warnings.filterwarnings(action="once")

#### Initialize the interactive segmentation class

As a first step, initialize the `tr.interactive.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

You will need to specify the following `args` (in order):


**Args**

**headpath (string)** : Top level path being used for processing (same as the rest of the mother machine pipeline.)

**seg_channel (string)** : String corresponding to the image channel in the input hdf5 file being used to segment the dataset.

In [ ]:
headpath = "/n/scratch2/de64/2019-05-13_bacillus_training_data"
seg_channel = "channel_RFP"

inter = tr.interactive.fluo_segmentation_interactive(headpath, seg_channel)

#### Import data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**fov_idx (int)** :

**n_trenches (int)** :

**t_range (tuple)** :

**t_subsample_step (int)** :

In [ ]:
kymo_arr_int = interactive(
    inter.import_array,
    {"manual": True},
    fov_idx=IntText(value=0, description="FOV number:", disabled=False),
    n_trenches=IntText(value=1, description="Number of trenches:", disabled=False),
    t_range=IntRangeSlider(
        value=[0, 100],
        description="Time Range:",
        min=0,
        max=200,
        step=1,
        disabled=False,
    ),
    t_subsample_step=IntSlider(
        value=1, description="Time Subsampling Step:", min=1, max=20, step=1
    ),
    fig_size_y=IntSlider(
        value=9, description="Figure Size (Y Dimension):", min=1, max=30, step=1
    ),
    fig_size_x=IntSlider(
        value=6, description="Figure Size (X Dimension):", min=1, max=30, step=1
    ),
)
display(kymo_arr_int)

In [ ]:
kymo_arr = copy.copy(kymo_arr_int.result)

#### Scale data

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**scale (bool)** : Whether to scale the kymograph in time.

**scaling_percentile (int)** : Whole image intensity percentile to use to determine scaling constant. 

In [ ]:
scaled_list_int = interactive(
    inter.plot_scaled,
    {"manual": True},
    kymo_arr=fixed(kymo_arr),
    scale=Dropdown(
        options=[True, False],
        value=True,
        description="Scale Fluorescence?",
        disabled=False,
    ),
    scaling_percentile=IntSlider(
        value=90,
        description="Scaling Percentile:",
        min=0,
        max=100,
        step=1,
        disabled=False,
    ),
)
display(scaled_list_int)


# scaled_list = inter.plot_scaled(kymo_arr,True,90)

In [ ]:
scaled_list = copy.copy(scaled_list_int.result)

#### Apply Gaussian Filter

Fill in 

You will need to tune the following `args` and `kwargs` (in order):

**smooth_sigma (float)** : Standard deviation of gaussian kernel.

In [ ]:
proc_list_int = interactive(
    inter.plot_processed,
    {"manual": True},
    scaled_list=fixed(scaled_list),
    smooth_sigma=FloatSlider(
        value=0.75,
        description="Gaussian Kernel Sigma:",
        min=0.0,
        max=3.0,
        step=0.25,
        disabled=False,
    ),
)
display(proc_list_int)

In [ ]:
proc_list = copy.copy(proc_list_int.result)
eigval_list = inter.plot_eigval(proc_list)

#### Determine Cell Mask Envelope

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**cell_mask_method (str)** : Thresholding method, can be a local or global Otsu threshold.

**cell_otsu_scaling (float)** : Scaling factor applied to determined threshold.

**local_otsu_r (int)** : Radius of thresholding kernel used in the local otsu thresholding.

In [ ]:
cell_mask_list_int = interactive(
    inter.plot_cell_mask,
    {"manual": True},
    proc_list=fixed(proc_list),
    cell_mask_method=Dropdown(
        options=["local", "global"],
        value="local",
        description="Cell Mask Thresholding Method:",
        disabled=False,
    ),
    cell_otsu_scaling=FloatSlider(
        value=0.85,
        description="Cell Threshold Scaling:",
        min=0.0,
        max=2.0,
        step=0.01,
        disabled=False,
    ),
    local_otsu_r=IntSlider(
        value=15,
        description="Local Otsu Radius:",
        min=0,
        max=30,
        step=1,
        disabled=False,
    ),
)
display(cell_mask_list_int)

In [ ]:
cell_mask_list = copy.copy(cell_mask_list_int.result)

#### Display Edge Mask at Threshold Value

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**edge_threshold_scaling (float)** : Scaling factor applied to determined threshold.

In [ ]:
composite_mask_list_int = interactive(
    inter.plot_threshold_result,
    {"manual": True},
    eigval_list=fixed(eigval_list),
    cell_mask_list=fixed(cell_mask_list),
    edge_threshold_scaling=FloatSlider(
        value=0.9,
        description="Edge Threshold Scaling",
        min=0.0,
        max=2.0,
        step=0.01,
        disabled=False,
    ),
)
display(composite_mask_list_int)

#### Threshold Sampling and Convexity Calculation

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**edge_threshold_scaling (float)** : Scaling factor applied to determined threshold.

**threshold_step_perc (float)** : Threshold step size to be used for trying multiple thresholds.

**threshold_perc_num_steps (int)** : Number of steps to use when generating multiple thresholds.

In [ ]:
conv_scores_list_int = interactive(
    inter.plot_scores,
    {"manual": True},
    eigval_list=fixed(eigval_list),
    cell_mask_list=fixed(cell_mask_list),
    edge_threshold_scaling=FloatSlider(
        value=0.9,
        description="Edge Threshold Scaling",
        min=0.0,
        max=2.0,
        step=0.01,
        disabled=False,
    ),
    threshold_step_perc=FloatSlider(
        value=0.05,
        description="Threshold Step Percent",
        min=0.0,
        max=0.5,
        step=0.01,
        disabled=False,
    ),
    threshold_perc_num_steps=IntSlider(
        value=2,
        description="Number of Threshold Steps",
        min=0,
        max=5,
        step=1,
        disabled=False,
    ),
)
display(conv_scores_list_int)

In [ ]:
conv_scores_list = copy.copy(conv_scores_list_int.result)

#### Convexity Thresholding

Fill in.

You will need to tune the following `args` and `kwargs` (in order):

**convex_threshold (float)** : Threshold to be used for convexity thresholding.

In [ ]:
final_mask_list_int = interactive(
    inter.plot_final_mask,
    {"manual": True},
    conv_scores_list=fixed(conv_scores_list),
    convex_threshold=FloatSlider(
        value=0.9,
        description="Convexity Threshold:",
        min=0.0,
        max=1.0,
        step=0.01,
        disabled=False,
    ),
)
display(final_mask_list_int)

In [ ]:
inter.print_results()